In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('s_and_p_quotes_hist.csv', parse_dates=['date'])

In [ ]:
cols = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']

In [ ]:
df.columns

In [ ]:
df = df[cols]

In [ ]:
df = df.dropna()

In [ ]:
def price_normalize(sym_df):
    sym_df.loc[:, 'year'] = sym_df.date.dt.year
    first_day = sym_df.groupby('year').first().reset_index()[['year', 'date', 'close']]
    first_day.columns = ['year', 'join_date', 'join_close']
    join_df = sym_df.merge(first_day, on='year')
    prc_cols = ['open', 'high', 'low', 'close']
    prc_cols_n = [x + '_n' for x in prc_cols]
    norm_df = join_df.loc[:, prc_cols] / join_df.loc[:, ['join_close']].values
    norm_df.columns = prc_cols_n
    out_df = pd.concat((join_df, norm_df), axis=1)
    out_df = out_df.loc[out_df['date'] != out_df['join_date']]
    return out_df

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_norm = df.groupby('symbol').apply(price_normalize)

In [ ]:
df_norm.to_parquet('historical_pr_data.parquet')

### Load Data From Parquet

In [1]:
import pandas as pd

df_norm = pd.read_parquet('historical_pr_data.parquet')

In [ ]:
sym_df = df_norm.loc[df_norm.symbol == 'A']

In [2]:
import ta

def get_indicators(sym_df, reversion_offsets=[30, 50, 100]):
    roc = ta.momentum.ROCIndicator(sym_df.close).roc()
    roc_short = ta.momentum.ROCIndicator(close=sym_df.close, n=6).roc()
    roc_long = ta.momentum.ROCIndicator(close=sym_df.close, n=18).roc()
    aws_ind = ta.momentum.AwesomeOscillatorIndicator(high=sym_df.high_n, low=sym_df.low_n).ao()
    mfi = ta.momentum.MFIIndicator(
        high=sym_df.high_n, low=sym_df.low_n,
        close=sym_df.close_n, volume=sym_df.volume
    ).money_flow_index()
    macd = ta.trend.MACD(close=sym_df.close_n).macd_diff()
    rsi = ta.momentum.RSIIndicator(close=sym_df.close_n).rsi()
    sym_df.loc[:, 'roc'] = roc
    sym_df.loc[:, 'roc_short'] = roc_short
    sym_df.loc[:, 'roc_long'] = roc_long
    sym_df.loc[:, 'aws'] = aws_ind
    sym_df.loc[:, 'mfi'] = mfi
    sym_df.loc[:, 'macd'] = macd
    sym_df.loc[:, 'rsi'] = rsi
    for offset in reversion_offsets:
        for meas in ['rsi', 'macd', 'mfi']:
            sym_df.loc[:, '%s_min_%d' % (meas, offset)] = sym_df[meas].rolling(offset, min_periods=1).min()
            sym_df.loc[:, '%s_max_%d' % (meas, offset)] = sym_df[meas].rolling(offset, min_periods=1).max()
    return sym_df

In [3]:
df_ind = df_norm.reset_index(drop=True).groupby('symbol').apply(get_indicators)

In [4]:
def get_entrances(df, lag_thresh=3,
              macd_oversold=-9e-3,
              rsi_oversold=30,
              mfi_oversold=30,
              popup=True):
    df.loc[:, 'macd_last'] = df.macd.shift(1)
    df.loc[:, 'aws_last'] = df.aws.shift(1)
    df.loc[:, 'close_last'] = df.close.shift(1)
    df.loc[:, 'macd_enter'] = 0
    macd_abv = df.macd > 0
    df.loc[(df.macd_last < 0) & macd_abv, 'macd_enter'] = 1
    lag_macd = df.macd_enter.rolling(lag_thresh, min_periods=1).max()
    df.loc[:, 'aws_enter'] = 0
    aws_abv = df.aws > 0
    df.loc[(df.aws_last < 0) & aws_abv, 'aws_enter'] = 1
    lag_aws = df.aws_enter.rolling(lag_thresh, min_periods=1).max()
    df.loc[macd_abv & (lag_macd == 1), 'macd_enter'] = 1
    df.loc[aws_abv & (lag_aws == 1), 'aws_enter'] = 1
    # The set criteria for these reversion based entrances are
    # mostly set based on desire for computational efficiency.
    if popup:
        popup_bool = df.close > df.close_last
    else:
        popup_bool = df.close == df.close
    df.loc[:, 'macd_oversold_enter'] = 0
    df.loc[(df.macd <= macd_oversold) & popup_bool, 'macd_oversold_enter'] = 1
    df.loc[:, 'rsi_oversold_enter'] = 0
    df.loc[(df.rsi <= rsi_oversold) & popup_bool, 'rsi_oversold_enter'] = 1
    df.loc[:, 'mfi_oversold_enter'] = 0
    df.loc[(df.mfi <= mfi_oversold) & popup_bool, 'mfi_oversold_enter'] = 1
    enter_cols = ['macd_enter', 'aws_enter', 'macd_oversold_enter', 'rsi_oversold_enter',
                  'mfi_oversold_enter']
    df.loc[:, 'entrances'] = df[enter_cols].sum(axis=1).clip(upper=1)
    return df

In [5]:
import numpy as np
import numpy.testing as tst

macd = [-5e-3, 5e-3, 5e-3, 5e-3, 5e-3, 5e-3, 5e-3, 5e-3, -9e-3, -9e-3,
        5e-3, 5e-3, 5e-3, 5e-3, -5e-3, 5e-3]
aws = [-5, -5, 5, 5, 5, 5, 5, 5, -5, -5, 5,
       5, 5, 5, -5, 5]
rsi = np.ones(len(aws)) * 60
rsi[2:5] = 30
mfi = np.ones(len(aws)) * 70
mfi[3:6] = 30
test_input_df = pd.DataFrame({'macd': macd, 'aws': aws, 'rsi': rsi, 'mfi': mfi, 'close': list(range(len(mfi)))})
test_input_df.loc[:, 'macd_last'] = test_input_df.macd.shift(1)
test_input_df.loc[:, 'aws_last'] = test_input_df.aws.shift(1)
macd_enter = [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1]
aws_enter = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1]
rsi_oversold_enter = np.zeros(len(rsi))
rsi_oversold_enter[2:5] = 1
macd_oversold_enter = np.zeros(len(macd))
macd_oversold_enter[8:10] = 1
mfi_oversold_enter = np.zeros(len(mfi))
mfi_oversold_enter[3:6] = 1
entrances = [0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
test_exp_out_df = pd.DataFrame({'macd_enter': macd_enter,
                                'aws_enter': aws_enter,
                                'macd_oversold_enter': macd_oversold_enter,
                                'rsi_oversold_enter': rsi_oversold_enter,
                                'mfi_oversold_enter': mfi_oversold_enter,
                                'entrances': entrances})
test_out_df = get_entrances(test_input_df, lag_thresh=3)
out_cols = ['macd_enter', 'aws_enter',
            'macd_oversold_enter', 'rsi_oversold_enter',
            'mfi_oversold_enter', 'entrances']
tst.assert_array_equal(test_out_df[out_cols].values, test_exp_out_df[out_cols].values)

In [6]:
df_entrance = df_ind.groupby('symbol').apply(lambda x: get_entrances(x, lag_thresh=1, popup=False))

In [7]:
open_prices =  [0.0,  0.8, 0.75, 0.77, 0.88, 0.90, 0.95,  1.0, 0.98,  0.91, 0.85, 0.87]
close_prices = [0.8, 0.75, 0.77, 0.88,  0.9, 0.95, 1.0, 0.98,  0.9, 0.85, 0.88,    0]
entrances =    [1.0,    0,    0,    0,    0,    0,   0,    0,    1,    0,    0,    0]
df_test = pd.DataFrame({'entrances': entrances, 'close': close_prices, 'open': open_prices})

In [8]:
from numba import jit


#@jit(nopython=True)
def backtest_numba(open_prices, close_prices, entrance, trail_stop, holding_per, prof_target=.20):
    n = len(entrance)
    profits = np.zeros(n)
    for ind1 in range(n - 1): # can't get next day open for last day
        if entrance[ind1] == 1:
            stop = int(min(n - ind1 - 1, holding_per))
            start_price = open_prices[ind1]
            peak_price = start_price
            open_profit = 0.0
            for ind2 in range(1, stop):
                current_price = close_prices[ind1 + ind2]
                open_price = open_prices[ind1 + ind2]
                if current_price > peak_price:
                    peak_price = current_price
                percentage_drop = (peak_price - current_price) / peak_price
                close_profit = current_price - start_price
                open_profit = open_price - start_price
                profit_perc = close_profit / start_price
                if percentage_drop >= trail_stop or profit_perc >= prof_target:
                    break
            profits[ind1] = open_profit
    return profits
                

def backtest(df, trail_stop=(0.1 - 1e-8), holding_per=30, prof_target=0.2):
    profits = backtest_numba(df.open.shift(-1).values, df.close.values,
                             df.entrances.values, trail_stop, holding_per,
                             prof_target=prof_target)
    df.loc[:, 'profits'] = profits
    df.loc[:, 'binary_profit'] = 0
    df.loc[df.profits > 0, 'binary_profit'] = 1
    return df

In [9]:
import numpy as np

df_out_test = backtest(df_test, prof_target=1.0)
exp_profits = np.array([0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.04, 0.0, 0.0, 0.0])
tst.assert_almost_equal(df_out_test.profits.values, exp_profits)

In [ ]:
df_out_test

In [10]:
%%time
df_profits = df_entrance.groupby('symbol').apply(lambda x: backtest(x, trail_stop=(0.2 - 1e-8), holding_per=50,
                                                                    prof_target=0.2))

Wall time: 37 s


In [ ]:
df_profits = df_entrance.groupby('symbol').apply(lambda x: backtest(x, trail_stop=(0.2 - 1e-8), holding_per=50,
                                                                    prof_target=1.0))

In [11]:
def get_profit_metrics(df_profits):
    prof_cols = []
    wins_losses = {}
    for col in out_cols:
        col_name = col + '_profits'
        df_profits.loc[:, col_name] = df_profits[col] * df_profits['profits']
        prof_cols.append(col_name)
        win_index = df_profits[col_name] > 0
        loss_index = df_profits[col_name] < 0
        mean_win = df_profits.loc[win_index, col_name].mean()
        mean_loss = df_profits.loc[loss_index, col_name].mean()
        sum_win = df_profits.loc[win_index, col_name].sum()
        sum_loss = df_profits.loc[loss_index, col_name].sum()
        win_perc = (df_profits.loc[win_index, col_name] /df_profits.loc[win_index, 'close']).mean()
        loss_perc = (df_profits.loc[loss_index, col_name] /df_profits.loc[loss_index, 'close']).mean()
        wins_losses[col_name] = [win_index.sum(), loss_index.sum(), mean_win, mean_loss, sum_win, sum_loss,
                                 win_perc, loss_perc]

    df_win_loss = pd.DataFrame(wins_losses, index=['wins', 'losses', 'mean_win',
                                                   'mean_loss', 'ttl_win', 'ttl_loss',
                                                   'mean_win_perc', 'mean_loss_perc']).transpose()
    df_win_loss.loc[:, 'win_loss_ratio'] =  df_win_loss.wins / df_win_loss.losses
    df_win_loss.loc[:, 'avg_w_l_prof_ratio'] = df_win_loss.mean_win / np.abs(df_win_loss.mean_loss)
    
    df_win_loss.loc[:, 'ttl_w_l_prof_ratio'] = df_win_loss.ttl_win / np.abs(df_win_loss.ttl_loss)
    df_win_loss.loc[:, 'net_profit'] = df_win_loss.ttl_win + df_win_loss.ttl_loss
    return df_win_loss

In [ ]:
win_loss_yr_metrics = df_profits.groupby('year').apply(get_profit_metrics)

In [12]:
df_win_loss = get_profit_metrics(df_profits)

In [ ]:
win_loss_yr_metrics.to_excel('metrics/baseline_performance.xlsx')

In [13]:
df_win_loss

,wins,losses,mean_win,mean_loss,ttl_win,ttl_loss,mean_win_perc,mean_loss_perc,win_loss_ratio,avg_w_l_prof_ratio,ttl_w_l_prof_ratio,net_profit
macd_enter_profits,25557.0,20326.0,4.000068,-4.976427,102229.7332,-101150.8533,0.105038,-0.106722,1.257355,0.803803,1.010666,1078.8799
aws_enter_profits,14543.0,11416.0,3.975075,-4.964310,57809.5087,-56672.5612,0.105522,-0.106175,1.273914,0.800731,1.020062,1136.9475
macd_oversold_enter_profits,68429.0,51873.0,4.597685,-5.276530,314614.9982,-273709.4504,0.137977,-0.126929,1.319164,0.871346,1.149449,40905.5478
rsi_oversold_enter_profits,43246.0,30320.0,4.287299,-4.700306,185408.5232,-142513.2921,0.115887,-0.110651,1.426319,0.912132,1.300991,42895.2311
mfi_oversold_enter_profits,64859.0,47300.0,4.101762,-4.541597,266036.2045,-214817.5425,0.123506,-0.115676,1.371226,0.903154,1.238429,51218.6620
entrances_profits,161164.0,121034.0,4.245874,-4.936122,684282.0562,-597438.5845,0.121322,-0.115866,1.331560,0.860164,1.145360,86843.4717


In [ ]:
df_win_loss

In [ ]:
def get_features(df):
    df.loc[:, 'close_n_last'] = df.close_n.shift(1)
    df.loc[:, 'close_diff'] = df.close_n - df.close_n_last
    df.loc[:, 'open_close_diff'] = df.open_n - df.close_n
    df.loc[:, 'high_low_diff'] = df.high_n - df.low_n
    return df

In [ ]:
df_profits = get_features(df_profits)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
input_cols = [
   'open_n', 'high_n', 'low_n', 'close_n',
   'roc', 'roc_short', 'roc_long', 'aws', 'mfi', 'macd', 'rsi',
   'rsi_min_30', 'rsi_max_30', 'macd_min_30', 'macd_max_30', 'mfi_min_30',
   'mfi_max_30', 'rsi_min_50', 'rsi_max_50', 'macd_min_50', 'macd_max_50',
   'mfi_min_50', 'mfi_max_50', 'rsi_min_100', 'rsi_max_100',
   'macd_min_100', 'macd_max_100', 'mfi_min_100', 'mfi_max_100',
   'macd_last', 'aws_last', 'close_last', 'macd_enter', 'aws_enter',
   'macd_oversold_enter', 'rsi_oversold_enter', 'mfi_oversold_enter',
   'entrances']
input_cols = ['roc', 'roc_short', 'roc_long', 'close_diff', 'open_close_diff', 'high_low_diff']
target_col = 'binary_profit'

In [ ]:
train_year_start = 2006
train_year_end = 2012

In [ ]:
start_year = 2013
end_year = 2013

In [ ]:
model_data = df_profits.loc[(df_profits.year != 2008) & (df_profits.profits != 0)]
train_data = model_data.loc[model_data.year.between(train_year_start, train_year_end), input_cols]
train_y = model_data.loc[model_data.year.between(train_year_start, train_year_end), target_col]
test = model_data.loc[model_data.year.between(start_year, end_year)]
test_orig = test.copy()
test_data = test.loc[:, input_cols]
test_y = test.loc[:, target_col]

In [ ]:
model = XGBClassifier(n_estimators=20, learning_rate=0.05, max_depth=5)

In [ ]:
model = model.fit(train_data, train_y,
                  eval_set=[(test_data, test_y)], verbose=True)

In [ ]:
preds = model.predict(test_data)

In [ ]:
test.loc[:, 'profits'] = test.profits * preds

In [ ]:
df_win_loss_ml = get_profit_metrics(test)

In [ ]:
df_win_loss = get_profit_metrics(test_orig)

In [ ]:
df_win_loss

In [ ]:
df_win_loss_ml

In [ ]:
df_profits.to_parquet('profits/profits_v1.parquet')

### Improvements 
* Look for price increase after reversion based indicators - V1
* Weight examples in XGBoost by Price. Look at training vs testing profit - V2
* Add Bollinger band based indicator - V3
* Add Regime Filter - V4
* Use Symbol Performance based filtering - V5
* Look at performance across Industry